In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import os

In [5]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [6]:
### dilation and erosion
def thin_font(image): # Erosion
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

def thick_font(image): # Dilation
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

In [7]:
def compute_lbp(image, radius=1, neighbors=8):
    # Convert the image to grayscale if it's in color
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Compute the LBP image
    lbp = np.zeros_like(gray)
    for i in range(len(gray)):
        for j in range(len(gray[0])):
            center = gray[i, j]
            pattern = 0
            for k in range(neighbors):
                x = i + int(round(radius * np.cos(2 * np.pi * k / neighbors)))
                y = j - int(round(radius * np.sin(2 * np.pi * k / neighbors)))
                if x >= 0 and x < len(gray) and y >= 0 and y < len(gray[0]):
                    pattern |= (gray[x, y] >= center) << k
            lbp[i, j] = pattern

    return lbp

In [5]:
training_defects = "./process_training/defects"
training_apples = "./process_training/apples"
validate_apples = "./process_validation/apples"
validate_defects = "./process_validation/defects"

# Output folder to save Color_segmentation images
output_train_defects = "./texture_training/defects"
output_train_apples = "./texture_training/apples"
output_valid_apples = "./texture_validation/apples"
output_valid_defects = "./texture_validation/defects"

# Process images in Training Defects
for filename in os.listdir(training_defects):
    img_path = os.path.join(training_defects, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = compute_lbp(img)
    output_path = os.path.join(output_train_defects, f'{filename}')
    cv2.imwrite(output_path, processed_img)

# Process images in Training Apples
for filename in os.listdir(training_apples):
    img_path = os.path.join(training_apples, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = compute_lbp(img)
    output_path = os.path.join(output_train_apples, f'{filename}')
    cv2.imwrite(output_path, processed_img)
    
# Process images in Validate Apples
for filename in os.listdir(validate_apples):
    img_path = os.path.join(validate_apples, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = compute_lbp(img)
    output_path = os.path.join(output_valid_apples, f'{filename}')
    cv2.imwrite(output_path, processed_img)
    
# Process images in Validate Defects
for filename in os.listdir(validate_defects):
    img_path = os.path.join(validate_defects, filename)
    img = cv2.imread(img_path)
#     thick_img = thick_font(img)
#     processed_img = thin_font(thick_img)
    processed_img = compute_lbp(img)
    output_path = os.path.join(output_valid_defects, f'{filename}')
    cv2.imwrite(output_path, processed_img)

In [8]:
train_dataset = train.flow_from_directory(
    './texture_training/',
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'binary'
)

validation_dataset = validation.flow_from_directory(
    './texture_validation/',
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'binary'
)

Found 456 images belonging to 2 classes.
Found 663 images belonging to 2 classes.


In [9]:
# To differentiate between classes
train_dataset.class_indices

{'apples': 0, 'defects': 1}

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(15, (3,3), activation='relu', input_shape=(256,256, 1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(30, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(60, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                                
])

In [17]:
# binary_crossentropy - binary classification (between the true labels and the predicted probabilities) 
# lr too hight may lead to instability or divergence - overshoot the minimum of the loss
# metric - track accuracy (the proportion of correctly classified samples)
model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
              metrics=['accuracy']
)

In [18]:
# steps_per_epoch = how many batches to process in each epoch
# lets take batch size as 100
# steps_per_epoch = total training data / batch size = 280 / 10 = 28
# the model will process 133 steps from the training dataset in each epoch
# epochs = train 30 times
model_fit = model.fit(
    train_dataset,
    steps_per_epoch = 28,
    epochs = 30,
    validation_data = validation_dataset
)

Epoch 1/30
28/28 [==============================] - 15s 485ms/step - loss: 0.0097 - accuracy: 0.9964 - val_loss: 2.5500 - val_accuracy: 0.5566
Epoch 2/30
28/28 [==============================] - 11s 402ms/step - loss: 0.1008 - accuracy: 0.9536 - val_loss: 1.9617 - val_accuracy: 0.4615
Epoch 3/30
28/28 [==============================] - 11s 409ms/step - loss: 0.0390 - accuracy: 0.9891 - val_loss: 1.3340 - val_accuracy: 0.5913
Epoch 4/30
28/28 [==============================] - 11s 411ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.6544 - val_accuracy: 0.5490
Epoch 5/30
28/28 [==============================] - 12s 429ms/step - loss: 3.9175e-04 - accuracy: 1.0000 - val_loss: 1.6542 - val_accuracy: 0.5505
Epoch 6/30
28/28 [==============================] - 12s 418ms/step - loss: 2.9580e-04 - accuracy: 1.0000 - val_loss: 1.7169 - val_accuracy: 0.5430
Epoch 7/30
28/28 [==============================] - 12s 423ms/step - loss: 3.2559e-04 - accuracy: 1.0000 - val_loss: 1.7356 - val_accu

In [19]:
model.save('./texture_apple.h5') # save performed morphology model

In [20]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
loaded_model = tf.keras.models.load_model('./texture_apple.h5')

def compute_lbp(image, radius=1, neighbors=8):
    # Convert the image to grayscale if it's in color
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Compute the LBP image
    lbp = np.zeros_like(gray)
    for i in range(len(gray)):
        for j in range(len(gray[0])):
            center = gray[i, j]
            pattern = 0
            for k in range(neighbors):
                x = i + int(round(radius * np.cos(2 * np.pi * k / neighbors)))
                y = j - int(round(radius * np.sin(2 * np.pi * k / neighbors)))
                if x >= 0 and x < len(gray) and y >= 0 and y < len(gray[0]):
                    pattern |= (gray[x, y] >= center) << k
            lbp[i, j] = pattern

    return lbp

# Function to preprocess the image for model input
def preprocess_img(edges):
    # Expand dimensions to match the expected input shape of the model
    edges = np.expand_dims(edges, axis=0)
    return edges

# Function to run prediction
def run_prediction(image_path):
    # Perform edge detection
    img = cv2.imread(image_path)
    img = cv2.resize(img, (256,256))
    edges = compute_lbp(img)
    
    # Preprocess the edges
    edges = preprocess_img(edges)
    
    # Make prediction using the loaded model
    prediction = loaded_model.predict(edges)
    
    # Interpret the prediction
    print(prediction)
    print(prediction[0])
    print(prediction[0][0])
    if prediction[0][0] == 0:
        print("This is an apple")
    else:
        print("This is a defect apple")

# Example usage:
image_path = './testData/21.jpg'
run_prediction(image_path)

1/1 [==============================] - 0s 172ms/step
[[0.]]
[0.]
0.0
This is an apple
